In [ ]:
with open('tokenfile', 'w') as f:
    f.write(dbutils.secrets.get(scope="creds", key="dbtoken"))
!databricks configure --host https://adb-7218252456053873.13.azuredatabricks.net/ --token-file tokenfile

In [ ]:
!pip install databricks-cli mlflow

  Using cached mlflow-2.7.1-py3-none-any.whl (18.5 MB)
  Using cached flask-2.3.3-py3-none-any.whl (96 kB)
  Using cached sqlparse-0.4.4-py3-none-any.whl (41 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached GitPython-3.1.40-py3-none-any.whl (190 kB)
  Using cached Markdown-3.5-py3-none-any.whl (101 kB)
  Using cached docker-6.1.3-py3-none-any.whl (148 kB)
  Using cached alembic-1.12.0-py3-none-any.whl (226 kB)
  Using cached SQLAlchemy-2.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached gunicorn-21.2.0-py3-none-any.whl (80 kB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
  Using cached websocket_client-1.6.4-py3-none-any.whl (57 kB)
  Using cached werkzeug-3.0.1-py3-none-any.whl (226 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Usin

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

In [ ]:
# Create an MLflow client
client = MlflowClient()

In [ ]:
# Get the latest version of the model from the experiment
model_name = "titanic_model"
model_version = client.search_model_versions(f"name='{model_name}'")[0].version

In [ ]:
model_version

'5'

In [ ]:
import requests
import json

# Define the API endpoint
url = "https://adb-7218252456053873.13.azuredatabricks.net/api/2.0/serving-endpoints"
token = dbutils.secrets.get(scope="creds", key="dbtoken")

# Define the headers for the request
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Define the data for the request
data = {
    "name": "titanic",
    "config": {
        "served_models": [
            {
                "model_name": model_name,
                "model_version": model_version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ]
    }
}

In [ ]:
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response
print(response.json())